In [3]:
import arcpy
import requests
import json
import pprint
import zipfile
from arcpy import env

#set working environment (where i want all my data to be saved to)
env.workspace = "D:\Spring_2022\AcGIS2\lab4\lab4_start1c"

## Attempting to compile accuracy assessment for all simulations
1. Calculate the confusion matrix ( True Positive (TP),False Positive (FP),False Negative (FN), True Negative (TN)) for each simulation
    
    a. Sum the TP, FP, FN, TN for ALL simulations

In [2]:
def identify_cities(feature_class, list_of_cols):
    '''
    Makes a dictionary that counts the number of city occurances within the data and returns the sum of the 4th column provided
    (can be used to return each cities ground truth from 2014-2020)
    ''' 
    count_rows = 0
    count_keep = 0
    d = dict()
    with arcpy.da.SearchCursor(feature_class, list_of_cols) as cursor: #may need to change the field names if different
        for point in cursor:
            count_rows += 1
            if point[2] not in d: #adding new cities to dictionary
                d[point[2]] = [1, point[3]] # value[0]=number of times city appears in data, value[1]= cities total population
                count_keep += 1 
            else: 
                for (key, value) in d.items():
                    if key ==point[2]:
                        #updateing the values to sum duplicate cities populations
                        d[point[2]] = [value[0] + 1, int(value[1]) + int(point[3])]  #counts the number of duplicates in dictionary
    return d


BMSB_cities_gtruth = identify_cities("BMSB_Pts_SJ", ['OID@', 'SHAPE@', 'CITY_SJ', 'G_Truth'])

print(len(BMSB_cities_gtruth))

157


In [4]:
#need to count the number of Presences (1) and Absences (0) recorded for each city over the 100 simulations
def Confusion_Sums(feature_class, list_of_cols, StinkBug_dict):
    '''
    This function counts the number of presence (1) and Absence for each city over 100 simulations. Based on these values we can 
    calculate TN, TP, FP, FN. We assign an Actual value to each city based on if it has recorded any presence of BMSB (from 2014 to 2020).
    It then updates the table to reflect the computed values for our confusion matrix.
    ''' 
    with arcpy.da.UpdateCursor(feature_class, list_of_cols) as cursor: #may need to change the field names if different
        for point in cursor:
            g_truth = None #restarting the values for each new row
            count_zeros = 0 
            count_ones = 0 
            #summing predictions from the 100 simulations
            for number in range(8,108):
                if point[number]==0:
                    count_zeros +=1
                elif point[number] ==1:
                    count_ones +=1
            if g_truth == None: 
                if point[2] not in StinkBug_dict:
                    g_truth = 0
                    point[4]=0
                    point[5]=int(count_ones)
                    point[6]=0
                    point[7]=int(count_zeros)
                    cursor.updateRow(point)
                elif point[2] in StinkBug_dict:
                    g_truth = int(StinkBug_dict[point[2]][1])
                    if g_truth == 0:
                        point[4]=0
                        point[5]=count_ones
                        point[6]=0
                        point[7]=count_zeros
                        cursor.updateRow(point)
                    elif g_truth >= 1:
                        point[4]=count_ones
                        point[5]=0
                        point[6]=count_zeros
                        point[7]=0
                        cursor.updateRow(point)
        print('Done!')
    del cursor
    
#making an accurate list of cols within 'MN_Cities_Pts_WGS'
list_of_fields = ['OBJECTID', 'SHAPE@', 'CITY_SJ', 'POPULATION', 'overall_TP', 'overall_FP', 'overall_FN', 'overall_TN']
for i in range(0,100):
    fieldname = "BMSB_Sim"+str(i)
    list_of_fields.append(fieldname)

#visually check attribute table for new values
Confusion_Sums("MN_Cities_Pts_WGS", list_of_fields, BMSB_cities_gtruth)

#note: i've been prompted with editing errors, need to use del cursor on any previous cursors used prior in notebook

Done!


In [5]:
#calculating accuracy 
def Accuracy(feature_class, list_of_cols):
    with arcpy.da.UpdateCursor(feature_class, list_of_cols) as cursor: #may need to change the field names if different
        for point in cursor:
            tp = point[4]
            fp = point[5]
            fn = point[6]
            tn = point[7]
            accur = (tp+tn)/(tp+tn+fp+fn)
            print(accur)
            point[8]=accur*100#current column stores whole numbers only
            cursor.updateRow(point)
    del cursor

Accuracy("MN_Cities_Pts_WGS", ['OBJECTID', 'SHAPE@', 'CITY_SJ', 'POPULATION', 'overall_TP', 'overall_FP', 'overall_FN', 'overall_TN', 'Accuracy'])

1.0
0.98
1.0
0.68
0.93
1.0
0.95
0.7
1.0
0.07
0.99
0.99
0.86
1.0
0.99
1.0
0.99
0.0
0.73
0.0
1.0
1.0
1.0
0.0
1.0
0.85
1.0
0.99
0.98
1.0
0.98
0.65
1.0
0.92
0.98
1.0
1.0
1.0
1.0
0.99
0.97
1.0
1.0
0.99
1.0
0.87
0.19
1.0
1.0
1.0
0.59
1.0
0.98
0.3
1.0
1.0
1.0
1.0
0.89
1.0
0.98
1.0
0.91
0.99
0.09
1.0
1.0
1.0
0.19
0.99
0.98
1.0
1.0
1.0
1.0
0.92
0.0
0.95
1.0
1.0
0.99
0.99
1.0
1.0
1.0
0.9
0.83
1.0
1.0
0.97
1.0
0.99
1.0
0.0
1.0
1.0
1.0
0.98
0.99
1.0
0.97
1.0
0.95
1.0
1.0
0.09
0.98
1.0
0.0
1.0
1.0
0.68
0.94
1.0
1.0
0.43
1.0
0.97
0.4
0.99
0.99
0.98
0.86
1.0
1.0
0.91
0.04
0.99
0.0
1.0
0.0
0.0
0.89
1.0
0.37
0.97
0.99
0.0
0.99
0.98
0.96
1.0
0.97
0.92
1.0
0.86
1.0
0.93
1.0
1.0
1.0
1.0
0.89
0.92
0.99
0.98
0.84
0.79
0.98
0.75
0.0
0.3
1.0
1.0
0.99
0.99
0.0
0.04
1.0
1.0
1.0
1.0
0.96
0.06
1.0
0.99
0.97
0.98
0.0
1.0
1.0
1.0
1.0
1.0
0.99
1.0
1.0
0.97
0.94
0.98
0.01
1.0
0.0
1.0
1.0
0.99
0.31
0.99
1.0
0.6
1.0
0.97
1.0
0.99
0.99
0.94
0.86
1.0
1.0
1.0
0.99
0.99
0.5
1.0
0.67
1.0
1.0
0.0
1.0
0.74
0.04
0.97
0.99
0.99

## RANKING ALGORITHM
Ranks cities based on FP (cities where our simulation predicted prsence (1) but has a ground truth of 0 (most likely due to lack of monitoring of BMSB in city since cities without current BSB monitering were assigned a gorund truth of 0) and the number of traps within a 10 mile radius of each city. 

This methods reduces the prioritizes cities with high FP and litllt to no BMSB traps in its vacinity. 


In [4]:
#spatial join to count traps within 10 miles of cities
# gets count of trap readings, not just geography
arcpy.analysis.SpatialJoin("MN_Cities_Pts_WGS", "BMSB_Pts_SJ", r"D:\Spring_2022\AcGIS2\lab4\lab4_start1c\lab4_start1c.gdb\BMSB_Cities_Join3", "JOIN_ONE_TO_ONE", "KEEP_ALL", 'CITY_SJ "CITY_SJ" true true false 512 Text 0 0,First,#,MN_Cities_WGS_Sorted,CITY_SJ,0,512;POPULATION "POPULATION" true true false 8 Double 0 0,First,#,MN_Cities_WGS_Sorted,POPULATION,-1,-1;ORIG_FID "ORIG_FID" true true false 4 Long 0 0,First,#,MN_Cities_WGS_Sorted,ORIG_FID,-1,-1;BMSB_Sim0 "BMSB_Sim0" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim0,-1,-1;BMSB_Sim1 "BMSB_Sim1" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim1,-1,-1;BMSB_Sim2 "BMSB_Sim2" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim2,-1,-1;BMSB_Sim3 "BMSB_Sim3" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim3,-1,-1;BMSB_Sim4 "BMSB_Sim4" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim4,-1,-1;BMSB_Sim5 "BMSB_Sim5" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim5,-1,-1;BMSB_Sim6 "BMSB_Sim6" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim6,-1,-1;BMSB_Sim7 "BMSB_Sim7" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim7,-1,-1;BMSB_Sim8 "BMSB_Sim8" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim8,-1,-1;BMSB_Sim9 "BMSB_Sim9" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim9,-1,-1;BMSB_Sim10 "BMSB_Sim10" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim10,-1,-1;BMSB_Sim11 "BMSB_Sim11" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim11,-1,-1;BMSB_Sim12 "BMSB_Sim12" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim12,-1,-1;BMSB_Sim13 "BMSB_Sim13" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim13,-1,-1;BMSB_Sim14 "BMSB_Sim14" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim14,-1,-1;BMSB_Sim15 "BMSB_Sim15" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim15,-1,-1;BMSB_Sim16 "BMSB_Sim16" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim16,-1,-1;BMSB_Sim17 "BMSB_Sim17" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim17,-1,-1;BMSB_Sim18 "BMSB_Sim18" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim18,-1,-1;BMSB_Sim19 "BMSB_Sim19" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim19,-1,-1;BMSB_Sim20 "BMSB_Sim20" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim20,-1,-1;BMSB_Sim21 "BMSB_Sim21" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim21,-1,-1;BMSB_Sim22 "BMSB_Sim22" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim22,-1,-1;BMSB_Sim23 "BMSB_Sim23" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim23,-1,-1;BMSB_Sim24 "BMSB_Sim24" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim24,-1,-1;BMSB_Sim25 "BMSB_Sim25" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim25,-1,-1;BMSB_Sim26 "BMSB_Sim26" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim26,-1,-1;BMSB_Sim27 "BMSB_Sim27" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim27,-1,-1;BMSB_Sim28 "BMSB_Sim28" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim28,-1,-1;BMSB_Sim29 "BMSB_Sim29" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim29,-1,-1;BMSB_Sim30 "BMSB_Sim30" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim30,-1,-1;BMSB_Sim31 "BMSB_Sim31" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim31,-1,-1;BMSB_Sim32 "BMSB_Sim32" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim32,-1,-1;BMSB_Sim33 "BMSB_Sim33" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim33,-1,-1;BMSB_Sim34 "BMSB_Sim34" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim34,-1,-1;BMSB_Sim35 "BMSB_Sim35" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim35,-1,-1;BMSB_Sim36 "BMSB_Sim36" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim36,-1,-1;BMSB_Sim37 "BMSB_Sim37" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim37,-1,-1;BMSB_Sim38 "BMSB_Sim38" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim38,-1,-1;BMSB_Sim39 "BMSB_Sim39" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim39,-1,-1;BMSB_Sim40 "BMSB_Sim40" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim40,-1,-1;BMSB_Sim41 "BMSB_Sim41" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim41,-1,-1;BMSB_Sim42 "BMSB_Sim42" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim42,-1,-1;BMSB_Sim43 "BMSB_Sim43" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim43,-1,-1;BMSB_Sim44 "BMSB_Sim44" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim44,-1,-1;BMSB_Sim45 "BMSB_Sim45" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim45,-1,-1;BMSB_Sim46 "BMSB_Sim46" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim46,-1,-1;BMSB_Sim47 "BMSB_Sim47" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim47,-1,-1;BMSB_Sim48 "BMSB_Sim48" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim48,-1,-1;BMSB_Sim49 "BMSB_Sim49" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim49,-1,-1;BMSB_Sim50 "BMSB_Sim50" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim50,-1,-1;BMSB_Sim51 "BMSB_Sim51" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim51,-1,-1;BMSB_Sim52 "BMSB_Sim52" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim52,-1,-1;BMSB_Sim53 "BMSB_Sim53" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim53,-1,-1;BMSB_Sim54 "BMSB_Sim54" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim54,-1,-1;BMSB_Sim55 "BMSB_Sim55" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim55,-1,-1;BMSB_Sim56 "BMSB_Sim56" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim56,-1,-1;BMSB_Sim57 "BMSB_Sim57" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim57,-1,-1;BMSB_Sim58 "BMSB_Sim58" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim58,-1,-1;BMSB_Sim59 "BMSB_Sim59" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim59,-1,-1;BMSB_Sim60 "BMSB_Sim60" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim60,-1,-1;BMSB_Sim61 "BMSB_Sim61" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim61,-1,-1;BMSB_Sim62 "BMSB_Sim62" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim62,-1,-1;BMSB_Sim63 "BMSB_Sim63" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim63,-1,-1;BMSB_Sim64 "BMSB_Sim64" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim64,-1,-1;BMSB_Sim65 "BMSB_Sim65" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim65,-1,-1;BMSB_Sim66 "BMSB_Sim66" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim66,-1,-1;BMSB_Sim67 "BMSB_Sim67" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim67,-1,-1;BMSB_Sim68 "BMSB_Sim68" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim68,-1,-1;BMSB_Sim69 "BMSB_Sim69" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim69,-1,-1;BMSB_Sim70 "BMSB_Sim70" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim70,-1,-1;BMSB_Sim71 "BMSB_Sim71" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim71,-1,-1;BMSB_Sim72 "BMSB_Sim72" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim72,-1,-1;BMSB_Sim73 "BMSB_Sim73" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim73,-1,-1;BMSB_Sim74 "BMSB_Sim74" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim74,-1,-1;BMSB_Sim75 "BMSB_Sim75" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim75,-1,-1;BMSB_Sim76 "BMSB_Sim76" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim76,-1,-1;BMSB_Sim77 "BMSB_Sim77" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim77,-1,-1;BMSB_Sim78 "BMSB_Sim78" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim78,-1,-1;BMSB_Sim79 "BMSB_Sim79" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim79,-1,-1;BMSB_Sim80 "BMSB_Sim80" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim80,-1,-1;BMSB_Sim81 "BMSB_Sim81" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim81,-1,-1;BMSB_Sim82 "BMSB_Sim82" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim82,-1,-1;BMSB_Sim83 "BMSB_Sim83" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim83,-1,-1;BMSB_Sim84 "BMSB_Sim84" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim84,-1,-1;BMSB_Sim85 "BMSB_Sim85" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim85,-1,-1;BMSB_Sim86 "BMSB_Sim86" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim86,-1,-1;BMSB_Sim87 "BMSB_Sim87" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim87,-1,-1;BMSB_Sim88 "BMSB_Sim88" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim88,-1,-1;BMSB_Sim89 "BMSB_Sim89" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim89,-1,-1;BMSB_Sim90 "BMSB_Sim90" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim90,-1,-1;BMSB_Sim91 "BMSB_Sim91" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim91,-1,-1;BMSB_Sim92 "BMSB_Sim92" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim92,-1,-1;BMSB_Sim93 "BMSB_Sim93" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim93,-1,-1;BMSB_Sim94 "BMSB_Sim94" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim94,-1,-1;BMSB_Sim95 "BMSB_Sim95" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim95,-1,-1;BMSB_Sim96 "BMSB_Sim96" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim96,-1,-1;BMSB_Sim97 "BMSB_Sim97" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim97,-1,-1;BMSB_Sim98 "BMSB_Sim98" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim98,-1,-1;BMSB_Sim99 "BMSB_Sim99" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,BMSB_Sim99,-1,-1;overall_TP "overall_TP" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,overall_TP,-1,-1;overall_FP "overall_FP" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,overall_FP,-1,-1;overall_FN "overall_FN" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,overall_FN,-1,-1;overall_TN "overall_TN" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,overall_TN,-1,-1;Accuracy "Accuracy" true true false 4 Float 0 0,First,#,MN_Cities_WGS_Sorted,Accuracy,-1,-1;Rank "Rank" true true false 2 Short 0 0,First,#,MN_Cities_WGS_Sorted,Rank,-1,-1;ORIG_FID_1 "ORIG_FID_1" true true false 4 Long 0 0,First,#,MN_Cities_WGS_Sorted,ORIG_FID_1,-1,-1;Join_Count "Join_Count" true true false 10 Long 0 10,Count,#,BMSB_Pts_SJ,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 10 Long 0 10,First,#,BMSB_Pts_SJ,TARGET_FID,-1,-1;City "City" true true false 254 Text 0 0,First,#,BMSB_Pts_SJ,City,0,254;Year "Year" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,Year,-1,-1;CheckDate "CheckDate" true true false 8 Date 0 0,First,#,BMSB_Pts_SJ,CheckDate,-1,-1;Adults "Adults" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,Adults,-1,-1;Nymphs "Nymphs" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,Nymphs,-1,-1;ALL "ALL" true true false 5 Long 0 5,First,#,BMSB_Pts_SJ,ALL,-1,-1;G_Truth "G_Truth" true true false 5 Long 0 5,First,#,BMSB_Pts_SJ,G_Truth,-1,-1;CITY_SJ_1 "CITY_SJ" true true false 254 Text 0 0,First,#,BMSB_Pts_SJ,CITY_SJ,0,254;POPULATION_1 "POPULATION" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,POPULATION,-1,-1;Shape_Leng "Shape_Leng" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,Shape_Leng,-1,-1;Shape_Area "Shape_Area" true true false 19 Double 0 0,First,#,BMSB_Pts_SJ,Shape_Area,-1,-1', "WITHIN_A_DISTANCE", "10 Miles", '')

#alter name of join count column to "Traps_Near"
arcpy.management.AlterField("BMSB_Cities_Join3", "Join_Count", "Traps_Near", "Traps_Near", "LONG", 4, "NULLABLE", "DO_NOT_CLEAR")

<Result 'BMSB_Cities_Join3'>

In [5]:
# Join "Traps_Near" column to MN_Cities_Pts_WGS table
arcpy.management.JoinField("MN_Cities_Pts_WGS", "CITY_SJ", "BMSB_Cities_Join3", "CITY_SJ", "Traps_Near")

<Result 'MN_Cities_Pts_WGS'>

In [9]:
def rankingIndex(feature_class):
    
    #add fields for priority ranking and ranking index
    arcpy.management.AddField("MN_Cities_Pts_WGS", "Priority", "SHORT", None, None, None, "Priority", "NULLABLE", "NON_REQUIRED", '')
    arcpy.management.AddField("MN_Cities_Pts_WGS", "RankIndex", "FLOAT", None, None, None, "RankIndex", "NULLABLE", "NON_REQUIRED", '')
    
    #create update cursor to run ranking algorithm
    with arcpy.da.UpdateCursor(feature_class, ['CITY_SJ','Overall_FP','Traps_Near','RankIndex','Priority']) as cursor:
        for row in cursor:
            
            FP = row[1]
            Traps = row[2]
            
            #ranking algorithm
            #can divide any cities with non zeros (non nulls) count of traps in a 10 mile radius
            # if Traps is not None: 
            if Traps != 0:
                row[3] = FP / Traps
            #if the count of TRaps is zero (null) assign it a low value of 0.5 to divide by which will still return a high number prioritizing these cities without traps
            else:
                row[3] = FP / 0.5
                
            cursor.updateRow(row)

rankingIndex('MN_Cities_Pts_WGS')

In [12]:
#new ranking algorithm
def rank_cities(feature_class):
    '''
    using sort to automatelically sort rows first based on RankIndex then on Population. Once it is sorted, use update cursor
    to populate rank column.
    '''
    #creating new table
    arcpy.management.Sort("MN_Cities_Pts_WGS", r"D:\Spring_2022\AcGIS2\lab4\lab4_start1c\lab4_start1c.gdb\MN_Cities_Pts_WGS_Sort2", [['RankIndex', 'DESCENDING'], ['POPULATION', 'DESCENDING']], "UR")
    #populating rank column
    with arcpy.da.UpdateCursor("MN_Cities_Pts_WGS_Sort2", ["Priority"]) as cursor:
        rank = 1
        for point in cursor:
            point[0] = rank
            cursor.updateRow(point)
            rank +=1 
    del cursor
    
rank_cities("MN_Cities_Pts_WGS")

## Overall Accuracy
average the accuracy column for model where alpha = 1.5 (0.8039), alpha =1.75 (0.8268), and alpha =2 (0.8430):

In [14]:
with arcpy.da.UpdateCursor("MN_Cities_Pts_WGS_Sort2", ["Accuracy"]) as cursor:
    count = 0
    Summ = 0
    for point in cursor:
        Summ += point[0]
        count +=1 
    average = Summ/count

print('Overall Accuracy: ', average)

Overall Accuracy:  0.8039624853458395


## summarize Presence column

In [20]:
#calculate the 'Presence' column Average, Min, max
with arcpy.da.SearchCursor("MN_Cities_Pts_WGS", ["Presence"]) as cursor:
    min_ = None
    max_ = None
    count = 0
    Summ = 0
    for point in cursor:
        Summ += point[0]
        count +=1
        if min_ == None:
            min_ = point[0]
        elif max_ == None:
            max_ = point[0]
        elif min_> point[0]:
            min_ = point[0]
        elif max_ < point[0]:
            max_ = point[0]
    average = Summ/count

print('Average Presence: ', average, 'Min:', min_, 'Max:', max_)

Average Presence:  0.2299648300117233 Min: 0.0 Max: 1.0


In [21]:
def hist_presence(feature_class, list_of_cols):
    '''
    Make a dictionary that counts the number of Presence occurances within the data and returns the sum of the 4th column provided
    (in this case the number of times each cities is predicted to have BMSB presence out of 100 simulations)
    ''' 
    count_rows = 0
    count_keep = 0
    d = dict()
    with arcpy.da.SearchCursor(feature_class, list_of_cols) as cursor: #may need to change the field names if different
        for point in cursor:
            count_rows += 1
            if point[3] not in d: #adding new cities to dictionary
                d[point[3]] = [1] # value[0]=number of times city appears in data, value[1]= cities total population
                count_keep += 1 
            else: 
                for (key, value) in d.items():
                    if key ==point[3]:
                        #updateing the values to sum duplicate cities populations
                        d[point[3]] = [value[0] + 1]  #counts the number of duplicates in dictionary
    return sorted(d.items())

hist_cities_presence = hist_presence("MN_Cities_Pts_WGS", ['OID@', 'SHAPE@', 'CITY_SJ', 'Presence'])

print(len(hist_cities_presence)) #returns a list

89


In [22]:
#returns the median of 'presence' column
count = 0
key1 = None
key2 = None
for item  in range(0,len(hist_cities_presence)):
#     print(hist_cities_presence[item])
    if count < 427: 
        count += hist_cities_presence[item][1][0]
        key1 = hist_cities_presence[item][0]
#     elif hist_cities_presence[item][1][0] > 427:
#         key2 = hist_cities_presence[item][0]
    elif count > 427:
        key2 = hist_cities_presence[item][0]
#         print(hist_cities_presence[item][0], key1, key2)
print(count, key1)

445 0.04
